In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from statsmodels.discrete.discrete_model import Probit #just to check the final correctness of algorithm
from scipy.stats import bernoulli

In [2]:
df = pd.read_csv('titanic.csv')

In [3]:
df

,Survived,(Intercept),Sexmale,Age,SibSp,Parch,Fare
0,0,1,1,22.0,1,0,7.2500
1,1,1,0,38.0,1,0,71.2833
2,1,1,0,26.0,0,0,7.9250
3,1,1,0,35.0,1,0,53.1000
4,0,1,1,35.0,0,0,8.0500
...,...,...,...,...,...,...,...
707,0,1,0,39.0,0,5,29.1250
708,0,1,1,27.0,0,0,13.0000
709,1,1,0,19.0,0,0,30.0000
710,1,1,1,26.0,0,0,30.0000


In [4]:
from sklearn.model_selection import train_test_split

In [5]:
df_in = df[['(Intercept)','Sexmale','Age','SibSp','Parch','Fare']].values
df_in
df_out = df[['Survived']].values
df_out
df_in

array([[ 1.    ,  1.    , 22.    ,  1.    ,  0.    ,  7.25  ],
       [ 1.    ,  0.    , 38.    ,  1.    ,  0.    , 71.2833],
       [ 1.    ,  0.    , 26.    ,  0.    ,  0.    ,  7.925 ],
       ...,
       [ 1.    ,  0.    , 19.    ,  0.    ,  0.    , 30.    ],
       [ 1.    ,  1.    , 26.    ,  0.    ,  0.    , 30.    ],
       [ 1.    ,  1.    , 32.    ,  0.    ,  0.    ,  7.75  ]])

In [6]:
x_train,x_test,y_train,y_test = train_test_split(df_in,df_out,random_state=100, test_size=0.2)

# Part 2

In [7]:
# x = df[['(Intercept)','Sexmale','Age','SibSp','Parch','Fare']].to_numpy()
# y = df['Survived'].to_numpy()
# x

In [8]:
yavg = np.average(y_train)
yavg

0.39718804920913886

# Getting the average of the input covariates

In [9]:
x_inter_avg = 1 #intercept column avg


In [10]:
x_sex_avg = np.average(x_train[:,1])  #sex column avg
x_sex_avg

0.6432337434094904

In [11]:
x_age_avg = np.average(x_train[:,2]) #age column avg
x_age_avg

29.481985940246044

In [12]:
#sibsp parch fare
x_sib_avg = np.average(x_train[:,3])  #sibsp column avg
x_sib_avg

0.5571177504393673

In [13]:
x_parch_avg = np.average(x_train[:,4]) #parch column avg
x_parch_avg

0.4024604569420035

In [14]:
x_fare_avg = np.average(x_train[:,5])  #fare column average
x_fare_avg

35.81783778558875

# Using the input covariates for guessing the inital value of vector of regression coefficients( for this each input is considered to be independent of the other)

In [15]:
bi = yavg/x_inter_avg
bi

0.39718804920913886

In [16]:
bs = yavg/x_sex_avg
bs

0.6174863387978142

In [17]:
ba = yavg/x_age_avg
ba

0.01347222843176704

In [18]:
bsi = yavg/x_sib_avg
bsi

0.7129337539432178

In [19]:
bpa = yavg/x_parch_avg
bpa

0.9868995633187774

In [20]:
bf = yavg/x_fare_avg
bf

0.011089112960608327

In [21]:
beta = np.array([bi,-bs,-ba,-bsi,-bpa,bf])
beta
x =x_train
y=y_train

In [22]:
class mle:
    def __init__(self,x,y,beta):
        self.x = x
        self.y =y
        self.beta = beta
    def normalcdf(self):
        return norm.cdf(self.x@self.beta.T)  #cdf of a normal distribution
    def normalpdf(self):
        return norm.pdf(self.x@self.beta.T) #pdf of a normal distribution
    def likelihood(self):
        cdf_val = self.normalcdf()
        n = self.y.size
        suml = 0
        for i in range(n):
            if cdf_val[i] != 1 and cdf_val[i]!=0:
                suml = suml + (y[i]*np.log(cdf_val[i]) + (1-y[i])*np.log(1-cdf_val[i]))  #maximum likelihood calculation
        return suml
    def gradient(self):
        cdf_val = self.normalcdf()
        pdf_val = self.normalpdf()
        n = self.y.size
        sumg =0
        for i in range(n):
            if cdf_val[i]!=1 and cdf_val[i]!=0:      #skipping the entries for which cdf is either exactly equal to 0 or 1
                first_term = y[i]*pdf_val[i]/cdf_val[i]
                second_term = (1-y[i])*pdf_val[i]/(1-cdf_val[i])
                sumg = sumg + (first_term -second_term)*x[i,:]
        return sumg  
    
    def hessian(self):
        cdf_val = self.normalcdf()
        pdf_val = self.normalpdf()
        n = self.y.size
        ans = np.empty(712, dtype=np.float64)        
        xc1 = np.empty([6,712], dtype=np.float64)
        xr1 = np.empty([712,6], dtype=np.float64)   #try
        term =np.empty(712, dtype=np.float64)        
        for i in range(n):
            if cdf_val[i]!=1 and cdf_val[i]!=0: #skipping the entries for which cdf is either exactly equal to 0 or 1
                xr = x[i,:]
                xc = xr.reshape(-1,1)
                xr = xc.reshape(1,-1)               
                first_term = -pdf_val[i]*y[i]*((x[i,:]@beta.T)*cdf_val[i] + pdf_val[i])/cdf_val[i]**2
                second_term = -pdf_val[i]*(1-y[i])*(pdf_val[i] - (x[i,:]@beta.T)*(1-cdf_val[i]))/(1-cdf_val[i])**2
                term[i] = (first_term +second_term)
                #print(first_term)
                ans[i] = term[i]
                xc1[:,i] = xc.flatten()
                xr1[i,:]= xr.flatten()   #try
        ans.flatten('F')            
        return (ans*xc1)@xr1
    

# Newton Raphson Algorithm for finding the vector of regression coefficients

In [23]:
def newton_raphson(mlemodel, tolerance=1e-3, maxiter=1000):

    i = 0
    err= 100      
    while i<maxiter and np.any(err > tolerance):
        
        
        gradient = mlemodel.gradient()
        hessian = mlemodel.hessian()         
        invhessian = np.linalg.inv(hessian)
        betaNew = mlemodel.beta - (np.linalg.inv(hessian) @ gradient)
        print(betaNew)
        err = betaNew - mlemodel.beta
        mlemodel.beta = betaNew            
        i += 1
        if i==1000:
            return mlemodel.beta
    return mlemodel.beta

In [24]:
mlemodel = mle(x,y,beta)


In [25]:
mlemodel.gradient()

array([ 134.98457536,  -10.64235857, 2901.06830771,  210.08447475,
        226.59371681, 8068.27025807])

In [26]:
mlemodel.hessian()

array([[-2.97781161e+02, -1.80044057e+02, -8.71093055e+03,
        -1.20937354e+02, -1.04467137e+02, -9.67343132e+03],
       [-1.80044057e+02, -1.80044057e+02, -5.39244388e+03,
        -4.58678693e+01, -3.31076775e+01, -4.82481415e+03],
       [-8.71093055e+03, -5.39244388e+03, -3.13807597e+05,
        -3.06838726e+03, -2.03543700e+03, -2.95943394e+05],
       [-1.20937354e+02, -4.58678693e+01, -3.06838726e+03,
        -1.99370271e+02, -9.48397726e+01, -7.14594423e+03],
       [-1.04467137e+02, -3.31076775e+01, -2.03543700e+03,
        -9.48397726e+01, -1.74140255e+02, -6.58797326e+03],
       [-9.67343132e+03, -4.82481415e+03, -2.95943394e+05,
        -7.14594423e+03, -6.58797326e+03, -8.60633539e+05]])

In [27]:
final =newton_raphson(mlemodel)

[ 0.50751909 -1.28737978 -0.00463237 -0.04467015  0.09726988  0.00609181]
[ 0.63570161 -1.45539692 -0.00585242 -0.09505646  0.05236611  0.00768352]
[ 0.71339318 -1.4870349  -0.00774143 -0.14622398  0.02975372  0.0085568 ]
[ 0.70735373 -1.51048803 -0.00709702 -0.17214071  0.00235428  0.00902868]


# Final vector of Regression Coefficients

In [28]:
final  #the final regression coefficients

array([ 0.70735373, -1.51048803, -0.00709702, -0.17214071,  0.00235428,
        0.00902868])

In [29]:
mlemodelfinal = mle(x,y,final) #using the mle class again to find the likelihood value

In [30]:
mlemodelfinal.likelihood() 

array([-280.51546907])

# Part 3

In [31]:
Jack = [1,1,20,0,0,7.5] 

In [32]:
Rose = [1,0,19,1,1,512]

In [33]:
norm.cdf(Jack@final)  #the probability value here shows that chances of Jack surviving are very less

0.19014566099789837

In [34]:
norm.cdf(Rose@final)   #the probability value here shows that there are high chances that Rose will survive

0.9999997488174193

In [35]:
print(Probit(y, x).fit().summary())  #Inbuilt Probit Regression to check the correctness of algorithm

Optimization terminated successfully.
         Current function value: 0.489501
         Iterations 6
                          Probit Regression Results                           
Dep. Variable:                      y   No. Observations:                  569
Model:                         Probit   Df Residuals:                      563
Method:                           MLE   Df Model:                            5
Date:                Fri, 16 Jul 2021   Pseudo R-squ.:                  0.2714
Time:                        00:16:00   Log-Likelihood:                -278.53
converged:                       True   LL-Null:                       -382.29
Covariance Type:            nonrobust   LLR p-value:                 6.990e-43
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7943      0.185      4.301      0.000       0.432       1.156
x1            -1.4943      0.

In [36]:
#pred_res =  1 if norm.cdf(x_test@final)>0.5 else 0

In [37]:
pred_res =np.empty([len(x_test),1])
for i in range(len(x_test)):
    print(x_test[i]@final)
    if norm.cdf(x_test[i]@final)>0.5:
        pred_res[i]=1
    else:
        pred_res[i]=0
    print(y_test[i],pred_res[i])

0.9650722721131316
[1] [1.]
1.06308434132654
[1] [1.]
-0.9978386320785845
[0] [0.]
0.4712826485356874
[1] [1.]
-0.8876903812874135
[0] [0.]
-0.9180958127448249
[0] [0.]
-1.0269715083400506
[0] [0.]
-0.8800494418837198
[1] [0.]
0.6543980235948401
[0] [1.]
-1.0870151793722278
[0] [0.]
0.5963412172722244
[1] [1.]
-0.9141468126209931
[0] [0.]
1.0976056798027467
[1] [1.]
0.33256882511436625
[0] [1.]
-0.8305221137029822
[0] [0.]
-0.8068100367955343
[0] [0.]
0.552091635198222
[1] [1.]
-1.1237685551060725
[0] [0.]
0.6351152441002739
[1] [1.]
-0.944238526689922
[0] [0.]
0.8433706691577485
[1] [1.]
0.6733914659909301
[1] [1.]
-0.9593259563181162
[1] [0.]
-0.9941412279224111
[0] [0.]
-0.9040799392269685
[1] [0.]
0.6950484988891699
[1] [1.]
0.3666962835991122
[1] [1.]
-0.8270508922685709
[0] [0.]
1.6764504072997592
[1] [1.]
-0.8739608336472361
[0] [0.]
0.8024549950673697
[1] [1.]
-0.9092712094270492
[0] [0.]
2.5113510384632534
[1] [1.]
-0.7024961726558524
[0] [0.]
0.616677165787888
[1] [1.]
-0.843

In [38]:
pred_res

array([[1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],

In [39]:
pred_res2 = norm.cdf(x_test@final)
pred_res2

array([0.83274569, 0.85612815, 0.15917881, 0.68128055, 0.18735366,
       0.17928435, 0.15221694, 0.18941626, 0.7435723 , 0.13851505,
       0.72452635, 0.18031985, 0.86381164, 0.63027011, 0.20312183,
       0.20988797, 0.70955721, 0.13055561, 0.73732337, 0.17252389,
       0.80048941, 0.74965086, 0.16869728, 0.16007706, 0.18297651,
       0.75648755, 0.64307721, 0.20410411, 0.953175  , 0.19106979,
       0.78885509, 0.18160349, 0.9939865 , 0.24118489, 0.73127616,
       0.19956484, 0.14203157, 0.20137134, 0.22591455, 0.1537498 ,
       0.77730775, 0.18441366, 0.78512549, 0.90954019, 0.89650997,
       0.16879409, 0.83057934, 0.7758101 , 0.97618097, 0.78420476,
       0.1534357 , 0.1758065 , 0.20533322, 0.81577644, 0.23365396,
       0.74941488, 0.17846157, 0.17597684, 0.21049017, 0.7431597 ,
       0.17065918, 0.14621671, 0.16173837, 0.18006906, 0.86727295,
       0.73588858, 0.73598043, 0.68337743, 0.16085502, 0.16534266,
       0.18491156, 0.70240368, 0.22378688, 0.15234717, 0.31340

In [40]:
from sklearn.metrics import accuracy_score

In [41]:
accuracy_score(y_test, pred_res)

0.8041958041958042

In [42]:
y_test.shape

(143, 1)